# OS Credential Dumping: NTDS - T1003.003 TH Notebook

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import timedelta 
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import pandas as pd

In [2]:
spark = SparkSession.builder \
    .appName("HELK Reader") \
    .master("spark://helk-spark-master:7077") \
    .config("spark.executor.memory","5g") \
    .enableHiveSupport() \
    .getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option("max_colwidth", None)

In [4]:
es_reader = (spark.read.format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.nodes","")
    .option("es.nodes.wan.only","true")
    .option("es.net.http.auth.user","elastic"))

In [5]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-2020.04.12")
app_df = es_reader.load("logs-endpoint-winevent-application-2020.04.12")

In [6]:
sysmon_df.createOrReplaceTempView("sysmon_events")
app_df.createOrReplaceTempView("application_events")

## Triger 1 - Find ntdsutil.exe execution

In [7]:
ntdsustil_log = sysmon_df.where((sysmon_df.process_name == "ntdsutil.exe") & (sysmon_df.event_id == '1')).select(F.col("process_name"),F.col("process_guid"),F.col("process_path"),F.col("process_command_line"),F.col("process_parent_name"),F.col("process_parent_guid"),F.col("process_parent_command_line"),F.col("user_name"),F.col("user_account"),F.col("user_logon_guid"),F.col("host_name"))

In [8]:
ntdsustil_log.toPandas()

,process_name,process_guid,process_path,process_command_line,process_parent_name,process_parent_guid,process_parent_command_line,user_name,user_account,user_logon_guid,host_name
0,ntdsutil.exe,CEEB91CE-FC86-5E92-0000-001079D2F31C,c:\windows\system32\ntdsutil.exe,"""c:\windows\system32\ntdsutil.exe"" ""ac i ntds"" ""ifm"" ""create full c:\temp"" q q",cmd.exe,CEEB91CE-FC52-5E92-0000-00101647F31C,"""c:\windows\system32\cmd.exe""",spadmin,research\spadmin,CEEB91CE-FB90-5E92-0000-00205E55E91C,dc1.research.com
1,ntdsutil.exe,CEEB91CE-FC83-5E92-0000-00105F94F31C,c:\windows\system32\ntdsutil.exe,"ntdsutil ""ac i ntds"" ""ifm"" ""create full c:\temp"" q q",cmd.exe,CEEB91CE-FC52-5E92-0000-00101647F31C,"""c:\windows\system32\cmd.exe""",spadmin,research\spadmin,CEEB91CE-FB90-5E92-0000-00207F55E91C,dc1.research.com
2,ntdsutil.exe,CEEB91CE-FC83-5E92-0000-00105CA6F31C,c:\windows\system32\ntdsutil.exe,"""c:\windows\system32\ntdsutil.exe"" ""ac i ntds"" ""ifm"" ""create full c:\temp"" q q",cmd.exe,CEEB91CE-FC52-5E92-0000-00101647F31C,"""c:\windows\system32\cmd.exe""",spadmin,research\spadmin,CEEB91CE-FB90-5E92-0000-00207F55E91C,dc1.research.com


In [9]:
event_11_sysmon = sysmon_df.where(sysmon_df.event_id == '11').groupby(sysmon_df.process_guid, sysmon_df.host_name).agg(F.collect_set("file_name").alias("files_created"),F.min('@timestamp').alias("min_time_files"),F.max('@timestamp').alias("max_time_files"))

In [10]:
event_12_sysmon = sysmon_df.where(sysmon_df.event_id == '12').groupby(sysmon_df.process_guid, sysmon_df.host_name).agg(F.collect_set("registry_key_path").alias("registry_createdDeleted"),F.min('@timestamp').alias("mintime_registry_createdDeleted"),F.max('@timestamp').alias("maxtime_registry_createdDeleted"))

In [11]:
event_13_sysmon = sysmon_df.where(sysmon_df.event_id == '13').groupby(sysmon_df.process_guid, sysmon_df.host_name).agg(F.collect_set("registry_key_path").alias("registry_valueSet"),F.min('@timestamp').alias("mintime_registry_valueSet"),F.max('@timestamp').alias("maxtime_registry_valueSet"))

In [12]:
event_14_sysmon = sysmon_df.where(sysmon_df.event_id == '14').groupby(sysmon_df.process_guid, sysmon_df.host_name).agg(F.collect_set("registry_key_path").alias("registry_rename"),F.min('@timestamp').alias("mintime_registry_rename"),F.max('@timestamp').alias("maxtime_registry_rename"))

In [13]:
ntdsustil_log_join_1 = event_11_sysmon.join(ntdsustil_log, on = ['process_guid','host_name'] ,how='right')

In [14]:
ntdsustil_log_join_2 = event_12_sysmon.join(ntdsustil_log_join_1, on= ['process_guid','host_name'] ,how='right')

In [15]:
ntdsustil_log_join_3 = event_13_sysmon.join(ntdsustil_log_join_2,on= ['process_guid','host_name'] ,how='right')

In [16]:
ntdsustil_log_join_4 = event_14_sysmon.join(ntdsustil_log_join_3,on= ['process_guid','host_name']  ,how='right')

In [17]:
ntdsustil_log_join_4.toPandas()

,process_guid,host_name,registry_rename,mintime_registry_rename,maxtime_registry_rename,registry_valueSet,mintime_registry_valueSet,maxtime_registry_valueSet,registry_createdDeleted,mintime_registry_createdDeleted,maxtime_registry_createdDeleted,files_created,min_time_files,max_time_files,process_name,process_path,process_command_line,process_parent_name,process_parent_guid,process_parent_command_line,user_name,user_account,user_logon_guid
0,CEEB91CE-FC83-5E92-0000-00105F94F31C,dc1.research.com,None,NaT,NaT,None,NaT,NaT,None,NaT,NaT,None,NaT,NaT,ntdsutil.exe,c:\windows\system32\ntdsutil.exe,"ntdsutil ""ac i ntds"" ""ifm"" ""create full c:\temp"" q q",cmd.exe,CEEB91CE-FC52-5E92-0000-00101647F31C,"""c:\windows\system32\cmd.exe""",spadmin,research\spadmin,CEEB91CE-FB90-5E92-0000-00207F55E91C
1,CEEB91CE-FC83-5E92-0000-00105CA6F31C,dc1.research.com,None,NaT,NaT,None,NaT,NaT,None,NaT,NaT,None,NaT,NaT,ntdsutil.exe,c:\windows\system32\ntdsutil.exe,"""c:\windows\system32\ntdsutil.exe"" ""ac i ntds"" ""ifm"" ""create full c:\temp"" q q",cmd.exe,CEEB91CE-FC52-5E92-0000-00101647F31C,"""c:\windows\system32\cmd.exe""",spadmin,research\spadmin,CEEB91CE-FB90-5E92-0000-00207F55E91C
2,CEEB91CE-FC86-5E92-0000-001079D2F31C,dc1.research.com,None,NaT,NaT,"[HKLM\System\CurrentControlSet\Services\VSS\Diag\VssapiPublisher\GETSTATE (Enter), HKLM\System\CurrentControlSet\Services\VSS\Diag\VssapiPublisher\PREPAREBACKUP (Leave), HKLM\System\CurrentControlSet\Services\VSS\Diag\VssapiPublisher\IDENTIFY (Leave), HKLM\System\CurrentControlSet\Services\VSS\Diag\VssapiPublisher\DOSNAPSHOT (Leave), HKLM\System\CurrentControlSet\Services\VSS\Diag\VssapiPublisher\PREPAREBACKUP (Enter), HKLM\System\CurrentControlSet\Services\VSS\Diag\VssapiPublisher\GETSTATE (Leave), HKLM\System\CurrentControlSet\Services\VSS\Diag\VssapiPublisher\BACKUPCOMPLETE (Leave), HKLM\System\CurrentControlSet\Services\VSS\Diag\VssapiPublisher\IDENTIFY (Enter), HKLM\System\CurrentControlSet\Services\VSS\Diag\VssapiPublisher\BACKUPCOMPLETE (Enter), HKLM\System\CurrentControlSet\Services\VSS\Diag\VssapiPublisher\DOSNAPSHOT (Enter)]",2020-04-12 11:33:27.462,2020-04-12 11:33:37.988,"[HKLM\System\CurrentControlSet\Services\VSS\Diag\VssapiPublisher, HKLM\System\CurrentControlSet\Services\Tcpip\Parameters]",2020-04-12 11:33:26.836,2020-04-12 11:33:26.968,"[c:\temp\active directory\ntds.dit, c:\temp\active directory, c:\temp\registry\security, c:\$snap_202004121433_volumec$, c:\temp\registry\system, c:\temp\registry, c:\temp, c:\users\spadmin\appdata\local\temp\tmp.edb]",2020-04-12 11:33:27.244,2020-04-12 11:33:44.461,ntdsutil.exe,c:\windows\system32\ntdsutil.exe,"""c:\windows\system32\ntdsutil.exe"" ""ac i ntds"" ""ifm"" ""create full c:\temp"" q q",cmd.exe,CEEB91CE-FC52-5E92-0000-00101647F31C,"""c:\windows\system32\cmd.exe""",spadmin,research\spadmin,CEEB91CE-FB90-5E92-0000-00205E55E91C


## Triger 2 -Find NTDS copies on  local disk

In [18]:
ntds_dc_app_log = spark.sql(
    '''
    SELECT host_name,param1,param5 
    FROM application_events 
    WHERE (event_id in (325, 326, 327)) and param1 LIKE "%NTDS" or param5 LIKE "%ntds.dit"
    ''')

In [19]:
ntds_dc_app_log.toPandas()

,host_name,param1,param5
0,dc1.research.com,lsass,\\?\GLOBALROOT\Device\HarddiskVolumeShadowCopy1\Windows\NTDS\ntds.dit
1,dc1.research.com,NTDS,C:\$SNAP_202004121433_VOLUMEC$\Windows\NTDS\ntds.dit
2,dc1.research.com,NTDS,c:\temp\Active Directory\ntds.dit
3,dc1.research.com,NTDS,c:\temp\Active Directory\ntds.dit
4,dc1.research.com,NTDS,C:\$SNAP_202004121433_VOLUMEC$\Windows\NTDS\ntds.dit


## Triger 3 - Find vssadmin.exe execution , Volume Shadow Copy activity

In [20]:
VSSadmin_sysmon = sysmon_df.filter( (sysmon_df.process_name == "vssadmin.exe") &
                                   (sysmon_df.event_id == '1') &
                                   ((sysmon_df.process_command_line.contains('create')) | (sysmon_df.process_command_line.contains('delete')))).select(F.col("process_name"),F.col("process_guid"),F.col("process_path"),F.col("process_command_line"),F.col("process_parent_name"),F.col("process_parent_guid"),F.col("process_parent_command_line"),F.col("user_name"),F.col("user_account"),F.col("user_logon_guid"),F.col("host_name"))

In [21]:
VSSadmin_sysmon.toPandas()

,process_name,process_guid,process_path,process_command_line,process_parent_name,process_parent_guid,process_parent_command_line,user_name,user_account,user_logon_guid,host_name
0,vssadmin.exe,CEEB91CE-0908-5E93-0000-0010214C1E1D,c:\windows\system32\vssadmin.exe,vssadmin delete shadows /shadow={f6217010-5c41-48ae-b37c-41b40311af09},cmd.exe,CEEB91CE-08AA-5E93-0000-0010D9BA1D1D,"""c:\windows\system32\cmd.exe""",sdadmin,research\sdadmin,CEEB91CE-05D9-5E93-0000-0020684D031D,dc1.research.com
1,vssadmin.exe,CEEB91CE-0898-5E93-0000-001022D4181D,c:\windows\system32\vssadmin.exe,vssadmin create shadow /for=c:,cmd.exe,CEEB91CE-0796-5E93-0000-0010F393131D,"""c:\windows\system32\cmd.exe""",sdadmin,research\sdadmin,CEEB91CE-05D9-5E93-0000-0020DA4E031D,dc1.research.com
2,vssadmin.exe,CEEB91CE-08AF-5E93-0000-001067CA1D1D,c:\windows\system32\vssadmin.exe,vssadmin create shadow /for=c:,cmd.exe,CEEB91CE-08AA-5E93-0000-0010D9BA1D1D,"""c:\windows\system32\cmd.exe""",sdadmin,research\sdadmin,CEEB91CE-05D9-5E93-0000-0020684D031D,dc1.research.com


In [22]:
VSSadmin_join_1 = event_11_sysmon.join(VSSadmin_sysmon, on = ['process_guid','host_name'] ,how='right')

In [23]:
VSSadmin_join_2 = event_12_sysmon.join(VSSadmin_join_1, on= ['process_guid','host_name'] ,how='right')

In [24]:
VSSadmin_join_3 = event_13_sysmon.join(VSSadmin_join_2, on= ['process_guid','host_name'] ,how='right')

In [25]:
VSSadmin_join_4 = event_14_sysmon.join(VSSadmin_join_3, on= ['process_guid','host_name']  ,how='right')

In [27]:
final_results = VSSadmin_join_4.toPandas()

In [28]:
final_results[["process_guid","host_name","process_name","process_path","process_command_line","process_parent_name","process_parent_guid","process_parent_command_line","user_account","user_logon_guid","registry_rename","registry_valueSet","registry_createdDeleted","files_created","registry_createdDeleted","registry_valueSet","registry_createdDeleted","files_created"]]

,process_guid,host_name,process_name,process_path,process_command_line,process_parent_name,process_parent_guid,process_parent_command_line,user_account,user_logon_guid,registry_rename,registry_valueSet,registry_createdDeleted,files_created,registry_createdDeleted,registry_valueSet,registry_createdDeleted,files_created
0,CEEB91CE-0898-5E93-0000-001022D4181D,dc1.research.com,vssadmin.exe,c:\windows\system32\vssadmin.exe,vssadmin create shadow /for=c:,cmd.exe,CEEB91CE-0796-5E93-0000-0010F393131D,"""c:\windows\system32\cmd.exe""",research\sdadmin,CEEB91CE-05D9-5E93-0000-0020DA4E031D,None,None,None,None,None,None,None,None
1,CEEB91CE-08AF-5E93-0000-001067CA1D1D,dc1.research.com,vssadmin.exe,c:\windows\system32\vssadmin.exe,vssadmin create shadow /for=c:,cmd.exe,CEEB91CE-08AA-5E93-0000-0010D9BA1D1D,"""c:\windows\system32\cmd.exe""",research\sdadmin,CEEB91CE-05D9-5E93-0000-0020684D031D,None,None,None,None,None,None,None,None
2,CEEB91CE-0908-5E93-0000-0010214C1E1D,dc1.research.com,vssadmin.exe,c:\windows\system32\vssadmin.exe,vssadmin delete shadows /shadow={f6217010-5c41-48ae-b37c-41b40311af09},cmd.exe,CEEB91CE-08AA-5E93-0000-0010D9BA1D1D,"""c:\windows\system32\cmd.exe""",research\sdadmin,CEEB91CE-05D9-5E93-0000-0020684D031D,None,None,None,None,None,None,None,None
